# <p style="text-align: center;">FIT5196 Assignment 2</p>

### Student Name : Audi Rivai
### Student ID       : 30399262

Last Updated: 14 October 2020

The content of this assignment are:
1. Introduction
2. Import Libraries
3. Loading Data
4. Handle Dirty Data
5. Handle Missing Data
6. Handle Outliers Data
7. Handle Dirty Is expedited delivery columns
7. Export Data
8. Summary

## 1. Introduction
This FIT5196 Assignment 2 will demostrate how we clean, check, handle the data such as missing data, dirty data, and outliers data. Using several method we will be explain the step and the reason why we are doing this step:

1. Explore the data to detect some errors or anomalies
2. Using linear model to detect 
3. Export the result into `.csv` and files.

in the data there are some variable that contain an error and need to be fix which are:

* <font color="blue">order_id</font>: A unique id for each order
* <font color="blue">customer_id</font>: A unique id for each customer 
* <font color="blue">date</font>: The date the order was made, given in YYYY-MM-DD format
* <font color="blue">nearest_warehouse</font>: A string denoting the name of the nearest warehouse to the customer
* <font color="blue">shopping_cart </font>: A list of tuples representing the order items: first element of the tuple is the item ordered, and the second element is the quantity ordered for such item. 
* <font color="blue">order_price</font>: A float denoting the order price in AUD. The order price is the price of items before any discounts and/or delivery charges are applied.
* <font color="blue">customer_lat</font>: Latitude of the customer’s location
* <font color="blue">customer_long</font>: Longitude of the customer’s location
* <font color="blue">coupon_discount</font>: An integer denoting the percentage discount to be applied to the order_price.
* <font color="blue">distance_to_nearest_warehouse</font>: e A float representing the arc distance, in kilometres, between the customer and the nearest warehouse to him/her. (radius of earth: 6378 KM)
* <font color="blue">delivery_charges</font>: A float representing the delivery charges of the order
* <font color="blue">order_total</font>: A float denoting the total of the order in AUD after all discounts and/or delivery charges are applied.
* <font color="blue">season</font>: A string denoting the season in which the order was placed. Refer to this link for details about how seasons are defined. 
* <font color="blue">is_expedited_delivery</font>: A boolean denoting whether the customer has requested an expedited delivery
* <font color="blue">latest_customer_review</font>: A string representing the latest customer review on his/her most recent order
* <font color="blue">is_happy_customer</font>: A boolean denoting whether the customer is a happy customer or had an issue with his/her last order.

## 2. Import Libraries

Environment: Python 3.7.0 and Jupyter notebook Libraries used
* re (for the regular expression, included in Anaconda Python 3.7)
* numpy and pandas for extract the data
* SentimentIntensityAnalyzer (to classify the text sentiment)
* seaborn and matplotlib.pyplot to visualize data
* sklearn linear model to create a model for making prediction
* itertools permutations for calculate and get the permutation

In [3]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
from itertools import permutations
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import seaborn as sns
from sklearn import linear_model

## 3. Loading Data

In this section there are 3 types of data that we want to solve the first is the dirty data. second is the outliers data and last is the missing data. we will look at the explanation about this data. we will read the csv using pandas `pd.read_csv()`

In [2]:
# read the dirty data
dirty_df = pd.read_csv("30399262_dirty_data.csv")
# list the columns
columns_dirty = list(dirty_df.columns)
dirty_df.head()

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer
0,ORD033408,ID0579410795,2019-03-30,Thompson,"[('iStream', 1), ('pearTV', 2), ('iAssist Line...",14995,76.82,-37.822192,144.968058,15,12822.57,Autumn,True,0.4214,renewal is great the price was perfect and it ...,True
1,ORD406627,ID0355818813,2019-11-10,Bakers,"[('iStream', 1), ('pearTV', 1)]",6460,97.49,-37.807381,144.998196,5,6234.49,Spring,True,0.3908,the best best phone ive owned,True
2,ORD128696,ID0332546101,2019-07-03,Thompson,"[('Candle Inferno', 2), ('iAssist Line', 2)]",5310,64.11,-37.806092,144.944799,15,4577.61,Winter,False,0.7593,great phone! awesome phone would definitely bu...,True
3,ORD033188,ID4353937734,2019-06-12,Bakers,"[('Olivia x460', 1), ('Toshika 750', 1)]",5545,72.59,-37.824925,145.011117,10,5063.09,autumn,False,2.1709,good great phone,True
4,ORD398245,ID4297530196,2019-11-29,Bakers,"[('Alcon 10', 1), ('Olivia x460', 2)]",11400,106.82,-37.801224,144.983865,10,10366.82,Spring,True,1.3975,great phone until i dropped it great phone unt...,True


as you can see in the dirty data has some anomalies that need to fix we will find the anomalies in very row and count how many error inside the dirty data

In [3]:
# read the outlier data
outliers_df = pd.read_csv("30399262_outlier_data.csv")
outliers_df.head()

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer
0,ORD348366,ID0060088408,2019-11-12,Nickolson,"[('Olivia x460', 2), ('Thunder line', 1)]",4630,156.735,-37.819338,144.983695,10,4323.735,Spring,True,1.2465,five stars love the phone it was flawless,True
1,ORD131648,ID3529934922,2019-11-05,Thompson,"[('pearTV', 2), ('Thunder line', 2)]",16980,86.600,-37.810039,144.904753,0,17066.600,Spring,False,3.7330,one star junk period,False
2,ORD455400,ID6167417948,2019-11-08,Nickolson,"[('Thunder line', 2), ('Universe Note', 2)]",11260,84.620,-37.814145,144.963143,15,9655.620,Spring,True,0.7503,wrong phone i ordered a white 128gb phone and ...,False
3,ORD127716,ID2392803738,2019-05-22,Nickolson,"[('iAssist Line', 2), ('pearTV', 1)]",10760,33.895,-37.809969,144.978517,0,10793.895,Autumn,False,1.2424,like new i bought this renewed but was very pl...,True
4,ORD315358,ID0589420528,2019-12-25,Nickolson,"[('Universe Note', 1), ('Alcon 10', 1), ('Oliv...",17075,74.440,-37.809694,144.972599,5,16295.690,Summer,False,1.0265,bueno like,True


an outlier is a data point that differs significantly from other observations. An outlier may be due to variability in the measurement or it may indicate experimental error. based on this data we will look which data is outliers and how we detect them.

In [4]:
# read the missing data
missing_df = pd.read_csv("30399262_missing_data.csv")
missing_df.head()

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer
0,ORD173027,ID5002595025,2019-08-13,NaN,"[('pearTV', 2), ('iAssist Line', 2)]",17070.0,62.72,-37.812548,144.953118,15,14572.22,Winter,False,NaN,nice good,1.0
1,ORD388411,ID6167229080,2019-03-03,Thompson,"[('Lucent 330S', 2), ('Thunder line', 2), ('Al...",24720.0,65.84,-37.821040,144.955238,25,18605.84,Autumn,True,1.1762,i purchased this to download books for school....,NaN
2,ORD152479,ID0296027679,2019-01-05,Thompson,"[('Universe Note', 2), ('iAssist Line', 2), ('...",12580.0,51.97,-37.811275,144.947170,15,10744.97,Summer,False,0.1559,scratches and too old device it has lot of scr...,0.0
3,ORD452973,ID1560149561,2019-09-27,Thompson,"[('pearTV', 1), ('Universe Note', 1)]",9760.0,100.36,-37.815400,144.937838,15,8396.36,Spring,True,0.8666,five stars excelent the best cell im front ecu...,1.0
4,ORD452188,ID4315295088,2019-05-31,Bakers,"[('Thunder line', 2), ('Olivia x460', 1)]",5585.0,79.71,-37.817322,144.991454,10,5106.21,Autumn,True,0.8805,all hail the king listen you get a phone that ...,1.0


missing data is where some value in every colomns is missing and we will fill the data based on the calculation from other model. we will also see the warehouse data 

In [5]:
# read warehouses data
warehouses_df = pd.read_csv("warehouses.csv")
# filter latitude and longitude
warehouse_lat = warehouses_df["lat"]
warehouse_lon = warehouses_df["lon"]
# get the name of the warehouse
warehouses = warehouses_df.names.tolist()
warehouses

FileNotFoundError: [Errno 2] No such file or directory: 'warehouses.csv'

we can see that there are three different warehouses which is 'Nickolson', 'Thompson', 'Bakers' we will use this for the dirty data and the missing data.

## 4. Handle Dirty Data

In this section we will be looking the process is clean the data. when researcher doing their analysis on their data, some data is not usually clean. Therefore, in order to clean them we will look some of the anomalies in the dirty data and fix them.

Fisrt thing is, we create a function for create a linear model.

In [6]:
# create a linear model based on the question
def linearModel(columns):
    # this function generate linear model for delivery charge
    # assign predictors
    x = columns[['distance_to_nearest_warehouse','is_expedited_delivery',
                 'is_happy_customer']]  
    # assign the target
    y = columns['delivery_charges']
    # generate the model
    delivery_model = linear_model.LinearRegression()
    delivery_model.fit(x, y)
    return delivery_model

### Handle Order_id

As you can see we will look at the `order_id` columns we will use `value_counts()` and search whether there is a duplicate or not

In [7]:
# search the order id anomalies that have more than 1 value
anom_id = dirty_df.order_id.value_counts() != 1
print("Anomalies Order Id:",len(list(anom_id[anom_id].index)))

Anomalies Order Id: 0


Since there are no relationship between `order_id` and other variables, therefore there are no anomalies and duplicate `order_id` in the dirty data. We can say that we do not need to fix them.

### Handle customer_id

`customer_id` is the id number for customers who make an order. some customer can purchases multiple times. Below we will look if there is a anomalies or not

In [8]:
# search the customer id anomalies that have more than 1 value
multiple_order_cust = dirty_df.customer_id.value_counts() != 1
multiple_id_cust = list(multiple_order_cust[multiple_order_cust].index)
print(multiple_id_cust)

['ID0777753012', 'ID6167489495', 'ID2385138108', 'ID6167440969', 'ID6167266651', 'ID0541122877', 'ID3208518274', 'ID0580036390', 'ID0492939880']


As you can see since the `customer_id` has no relationship with other variables or columns and some people can make multiple order, we can tell that this is not an anomalies

### Handle date

we can see from the description the format of the date is `YYYY-MM-DD` the problem of this columns is:

* Find the date that have different format
* Check the day and month, are they get switch

we will check every row and demonstrate how to fix them. we will use `re` packages to print the date pattern and fix them
`[0-9]{4}-[0-9]{1,2}-[0-9]{1,2}` the pattern is to get the `YYYY-MM-DD` format. another mistake in the `date` is that the month of the date is in the wrong place.

In [ ]:
# change the date as a string type
dirty_df['date'] = dirty_df['date'].astype("string")

In [ ]:
# create an empty list
wrong_Date_For = []
# change the data into a list
date = dirty_df['date'].tolist()

# Defines a Regular Expression that depicts a YYYY-MM-DD format.
patternName = "[0-9]{4}-[0-9]{1,2}-[0-9]{1,2}"

# loop for every row to see the pattern
for index in range(len(date)):
    wrong = re.findall(patternName, dirty_df['date'].iloc[index]) 
    
    # If there is no match, it means the date is incorrectly formatted.
    if len(wrong) == 0:
        wrong_Date_For.append(dirty_df['date'].iloc[index])
    else:
        continue

print(wrong_Date_For)
print("incorrect date format: ", len(wrong_Date_For))

In [ ]:
# create empyt list
wrong_month_For = []
# change the data into a list
date2 = dirty_df['date'].tolist()

# This is able to find dates in the YYYY-DD-MM format by checking if the number 
# in the 'DD' field is > 12.
for index in range(len(date2)):
    if int(date2[index][5:7]) > 12:
        wrong_month_For.append(date2[index])
    else:
        continue

print(wrong_month_For)
print("incorrect month format: ", len(wrong_month_For))

Based on the result from above, there are 8 incorrect pattern in the `date` columns and 19 incorrect month location. to fix this problem we will use a loop the get the date in the right place and in the right format.

In [ ]:
## https://www.geeksforgeeks.org/python-program-to-swap-two-elements-in-a-list/

# create empyt list
clean_date = []

# split the date when they have "-"
for i in dirty_df['date'].str.split("-"):
    # if year lenght is not detected switch the value between day and year
    if len(i[0]) != 4:
        i[0], i[2] = i[2], i[0]
    # if month is not detected the switch the value between day and month
    if int(i[1]) > 12:
        i[1], i[2] = i[2], i[1]
    # join them again
    i = '-'.join(i)
    # append the result 
    clean_date.append(i)

# change the value to the date columns
dirty_df['date'] = clean_date
# change the date column into a datetime type
dirty_df['date']= pd.to_datetime(dirty_df['date'])
# extract the month 
dirty_df['month'] = dirty_df['date'].dt.month
# change the date as a string type
dirty_df['date'] = dirty_df['date'].astype(str)

Using this code from above, this will able to fix the date problem. After fix the code we will try to check using the same code from the previous cells and see whether there is another anomalies

In [ ]:
correct_Date_For = []
date = dirty_df['date'].tolist()

# Defines a Regular Expression that depicts a YYYY-MM-DD format.
patternName = "[0-9]{4}-[0-9]{1,2}-[0-9]{1,2}"

for index in range(len(date)):
    wrong = re.findall(patternName, dirty_df['date'].iloc[index]) 
    
    # If there is no match, it means the date is incorrectly formatted.
    if len(wrong) == 0:
        correct_Date_For.append(dirty_df['date'].iloc[index])
    else:
        continue

print(correct_Date_For)
print("incorrect date format: ", len(correct_Date_For))

In [ ]:
correct_month_For = []
date2 = dirty_df['date'].tolist()

# This is able to find dates in the YYYY-DD-MM format by checking if the number 
# in the 'DD' field is > 12.
for index in range(len(date2)):
    if int(date2[index][5:7]) > 12:
        correct_month_For.append(date2[index])
    else:
        continue

print(correct_month_For)
print("incorrect month format: ", len(correct_month_For))

In [ ]:
anom_date = len(wrong_month_For) + len(wrong_Date_For)
print("Total anomalies row in date columns:", anom_date)

Based on the result there are no `Syntactical Anomalies` in the date problem

### Handle season

In this part we want to check the anomalies of the data in the `season` columns. The the specific problem is:

* Naming problem
* is the season match with the month

In [ ]:
# print the value count to check the value
print(dirty_df["season"].value_counts())

# get the anomalies season
win_list_index = list(dirty_df[dirty_df["season"] == "winter"].index)
sum_list_index = list(dirty_df[dirty_df["season"] == "summer"].index)
spr_list_index = list(dirty_df[dirty_df["season"] == "spring"].index)
aut_list_index = list(dirty_df[dirty_df["season"] == "autumn"].index)

name_anomalies_season = win_list_index + sum_list_index +\
    spr_list_index + aut_list_index

In [ ]:
# change the naming season in the data
dirty_df.season.replace({"winter": "Winter", "summer":"Summer",
                         "spring":"Spring", "autumn":"Autumn"},
                        inplace=True)

# print the value count and check how many anomalies
print(dirty_df["season"].value_counts())
print("naming season anomalies: ", len(name_anomalies_season))

As you can see there are 22 anomalies for the naming in the date columns. next we will check the season is match with the month.

In Australia, the seasons are defined by grouping the calendar months in the following way:

* Spring - the three transition months September, October and November.
* Summer - the three hottest months December, January and February.
* Autumn - the transition months March, April and May.
* Winter - the three coldest months June, July and August.

http://www.bom.gov.au/climate/glossary/seasons.shtml

In [ ]:
# set up the month in every season
summer = [12, 1, 2]
autumn = [3, 4, 5]
winter = [6, 7, 8]
spring = [9, 10, 11]

# use group by to count each season and 
# to check is the season is in the right month
check_season = dirty_df.groupby(['season','month']).count()
# print the result
print(check_season.index)

# check how many wrong in each season
wrong_summer = list(np.where((dirty_df["season"] == "Summer") & 
                             (~dirty_df['month'].isin(summer)))[0])
wrong_autumn = list(np.where((dirty_df["season"] == "Autumn") & 
                             (~dirty_df['month'].isin(autumn)))[0])
wrong_winter = list(np.where((dirty_df["season"] == "Winter") & 
                             (~dirty_df['month'].isin(winter)))[0])
wrong_spring = list(np.where((dirty_df["season"] == "Spring") & 
                             (~dirty_df['month'].isin(spring)))[0])

# Calculate the total mistake in the season
wrong_season_month = wrong_summer + wrong_autumn + wrong_winter + wrong_spring
print("incorrect season based on month:", len(wrong_season_month))

In [ ]:
# use location that contain the correct month and change the season name
dirty_df.loc[dirty_df['month'].isin([12, 1, 2]), "season"] = "Summer"
dirty_df.loc[dirty_df['month'].isin([3, 4, 5]), "season"] = "Autumn"
dirty_df.loc[dirty_df['month'].isin([6, 7, 8]), "season"] = "Winter"
dirty_df.loc[dirty_df['month'].isin([9, 10, 11]), "season"] = "Spring"

# group by to check the result
dirty_df.groupby(['season','month']).count().index

In [ ]:
anom_season = list(set(name_anomalies_season + wrong_season_month))
print("total anomalies row in season columns:" ,len(anom_season))

we can tell that there are some mistake in the naming and the month season. for example, autumns suppose to be in the March, April and May, therefore we just need to change the value based on the correct month. some index have both anomalies so we have 27 row anomalies

### Handle customer_lat and customer_long

every customer have different coordinate which is the latitude and the longitude. to check the coordinate we will see:

* plot the data using package and explore the data
* check if the coordinate get switch between latitude and longitude

In [ ]:
# plot the histogram of the data latitude
plt.hist(dirty_df['customer_lat'])
plt.title('Histogram of Customer Latitude')
plt.xlabel('Customer Latitude')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# count how many anomalies that have more than 0
anom_lat = list(dirty_df[dirty_df['customer_lat']>0]['customer_lat'].index)
print("incorrect latitude:", len(anom_lat))

In [ ]:
# plot the histogram of the data longitude
plt.hist(dirty_df['customer_long'])
plt.title('Histogram of Customer Longitude')
plt.xlabel('Customer Longitude')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# count how many anomalies that have less than 0
anom_long = list(dirty_df[dirty_df['customer_long']<0]['customer_long'].index)
print("incorrect longitude:", len(anom_long))

we can see that the problem for the coordinate is some of the latitude and longitude are get switch. therefore we will get the index and switch them back in order to have the right coordinate.

In [ ]:
# Cast both these columns into respective lists.
custLat = dirty_df['customer_lat'].tolist()
custLon = dirty_df['customer_long'].tolist()

for index in range(len(custLat)): 
    # This fixes the second error in which the values are swapped.
    if custLat[index] > 0 and custLon[index] < 0:
        dirty_df.iloc[index, 7] = custLon[index]
        dirty_df.iloc[index, 8] = custLat[index]

In [ ]:
# plot the result using histogram to check whether there is a mistake
plt.hist(dirty_df['customer_lat'])
plt.title('Histogram of Customer Latitude')
plt.xlabel('Customer Latitude')
plt.ylabel('Frequency')
plt.show()

plt.hist(dirty_df['customer_long'])
plt.title('Histogram of Customer Longitude')
plt.xlabel('Customer Longitude')
plt.ylabel('Frequency')
plt.show()

print("incorrect latitude:", 
      len(list(dirty_df[dirty_df['customer_lat']>0]['customer_lat'])))
print("incorrect longitude:", 
      len(list(dirty_df[dirty_df['customer_long']<0]['customer_long'])))

In [ ]:
anom_coor = list(set(anom_lat + anom_long))
print("Total Anomalies coordinate:" , len(anom_coor))

We can see that the new coordinates is in the correct histogram boundary

### Handle distance_to_nearest_warehouse and nearest_warehouses

as you can see from the `warehouses.csv` there are three type of warehouses, in this section we will look at the nearest warehouse columns and found the anomalies. There are view thing that need to be check:

* Naming problem: is there any incorrect naming in the column's value
* Check the location and see if the warehouse is in the right place

After we check the incorrect naming in the `nearest_warehouses`, however there are some anomalise. this anomalies can be called. `Semantic Anomalies` hinder the data collection from being a comprehensive and non-redundant representation of the mini-world. These types of anomalies include: Integrity constraint violations, contradictions, duplicates and invalid tuples
this anomalies also involve in the `distance_to_nearest_warehouse`

to find this kind of anomalies in this section we will look at:


* using the warehouse data create a distance function, to calculate all the distance and check the distance and warehouse

In [ ]:
# see the value of the nearest warehouse
print(dirty_df.nearest_warehouse.value_counts())
# filter the incorrect name and calculate how many of them
nic_list_index=list(dirty_df[dirty_df[
    "nearest_warehouse"] == "nickolson"].index)
tho_list_index=list(dirty_df[dirty_df[
    "nearest_warehouse"] == "thompson"].index)
bac_list_index=list(dirty_df[dirty_df[
    "nearest_warehouse"] == "bakers"].index)

name_anomalies_wr = nic_list_index + tho_list_index + bac_list_index
print("naming warehouses anomalies: ", len(name_anomalies_wr))

Based from the table we can see there are some naming problem and we will fix this and see how many mistake the data have.

In [ ]:
# use replace to change the name
dirty_df.nearest_warehouse.replace({"nickolson": "Nickolson", 
                                    "thompson":"Thompson", 
                                    "bakers":"Bakers"}, 
                                   inplace=True)

# print the value count to check them
print(dirty_df.nearest_warehouse.value_counts())

This anomali is called `Syntactical Anomalies`, it is describe characteristics concerning the format and values used for representation of the entities. Syntactic anomalies such as: lexical errors, domain format errors, syntactical error and irregularities. to solve this problem we will use `replace()` to change the naming problem.

After this we will check the place of the warehouse. however we need to check the coodinate and the distance first in the next section.

In [ ]:
# create a coordinates based on the warehouse place
Nickolson_coor = warehouses_df.iloc[0][["lat", "lon"]]
Thompson_coor = warehouses_df.iloc[1][["lat", "lon"]]
Bakers_coor = warehouses_df.iloc[2][["lat", "lon"]]

In [ ]:
# https://stackoverflow.com/questions/33029396/
# using-pandas-to-calculate-distance-between-coordinates-from-imported-csv

# determine the radius
R = 6378

# create a function that calculate the coordinates 
# between the customer and the warehouse
def dist_places_warehouse(lat1, lon1, lat2, lon2):
    
    d_lat = np.radians(lat2-lat1)
    d_lon = np.radians(lon2-lon1)
    r_lat1 = np.radians(lat1)
    r_lat2 = np.radians(lat2)
    
    a = np.sin(d_lat/2.) **2 +\
        np.cos(r_lat1) * np.cos(r_lat2) * np.sin(d_lon/2.)**2
    haversine = 2 * R * np.arcsin(np.sqrt(a))
    
    return haversine

In [ ]:
# create a new columns that contain distance 
# from customer coordinate and warehouses
dirty_df["distance_to_Nickolson"] = dirty_df.apply(lambda x : 
                                                   dist_places_warehouse(
                                                    x["customer_lat"], 
                                                    x["customer_long"], 
                                                    Nickolson_coor["lat"],
                                                    Nickolson_coor["lon"]), 
                                                   axis=1).round(4)

dirty_df["distance_to_Thompson"] = dirty_df.apply(lambda x : 
                                                  dist_places_warehouse(
                                                    x["customer_lat"], 
                                                    x["customer_long"], 
                                                    Thompson_coor["lat"],
                                                    Thompson_coor["lon"]), 
                                                  axis=1).round(4)

dirty_df["distance_to_Bakers"] = dirty_df.apply(lambda x : 
                                                dist_places_warehouse(
                                                  x["customer_lat"], 
                                                  x["customer_long"], 
                                                  Bakers_coor["lat"],
                                                  Bakers_coor["lon"]), 
                                                axis=1).round(4)

In [ ]:
# create a empty list for monitoring the index 
# and the incorrect data
incorrect_distance = []
index_incorrect_distance = []
incorrect_warehouse = []
index_incorrect_warehouse = []
min_nearest = []
correct_warehouse = []

# list the warehouse place
warehouse_places = list(warehouses_df["names"])

# loop every row and calculate the distance then check the value
for h in range(len(dirty_df)):
    dist_near = round(dirty_df["distance_to_nearest_warehouse"][h],
                      4)
    wr_place = dirty_df["nearest_warehouse"][h]
    list_wr = [dirty_df["distance_to_Nickolson"][h],
               dirty_df["distance_to_Thompson"][h],
               dirty_df["distance_to_Bakers"][h]]
    # find the minimum distance
    min_dist_idx = list_wr.index(min(list_wr))
    correct_warehouse.append(warehouse_places[min_dist_idx])
    min_nearest.append(min(list_wr))
    # if the warehouse is not the same append 
    # the value index and value to the list
    if wr_place != warehouse_places[min_dist_idx]:
        incorrect_warehouse.append(wr_place)
        index_incorrect_warehouse.append(h)
    # if the distance warehouse is not the same append
    # the value index and value to the list
    if dist_near not in list_wr:
        incorrect_distance.append(dist_near)
        index_incorrect_distance.append(h)

        
print("Incorrect distance:", len(incorrect_distance),
      "\nIncorrect nearest warehouses:", len(incorrect_warehouse))  

using the code there are some row that have incorrect distance warehouse and the place of the warehouses. to fix the error, we will look at the index and changes the value.

In [ ]:
# list the correct value
list_incorrect_warehouse = [min_nearest[i] for i in 
                            index_incorrect_distance]
list_incorrect_distance = [correct_warehouse[i] for i 
                           in index_incorrect_warehouse]

# put the result into the data based on the index
dirty_df.loc[index_incorrect_warehouse,
             "nearest_warehouse"] = list_incorrect_distance
dirty_df.loc[index_incorrect_distance,
             "distance_to_nearest_warehouse"] = list_incorrect_warehouse

In [ ]:
# use the same code to check the value
new_correct_distance = []
index_new_correct_distance = []
new_correct_warehouse = []
index_new_correct_warehouse = []
min_nearest = []
correct_warehouse = []
warehouse_places = list(warehouses_df["names"])


for h in range(len(dirty_df)):
    dist_near = round(dirty_df["distance_to_nearest_warehouse"][h],
                      4)
    wr_place = dirty_df["nearest_warehouse"][h]
    list_wr = [dirty_df["distance_to_Nickolson"][h],
               dirty_df["distance_to_Thompson"][h],
               dirty_df["distance_to_Bakers"][h]]
    min_dist_idx = list_wr.index(min(list_wr))
    correct_warehouse.append(warehouse_places[min_dist_idx])
    min_nearest.append(min(list_wr))
    if wr_place != warehouse_places[min_dist_idx]:
        new_correct_warehouse.append(wr_place)
        index_new_correct_warehouse.append(h)
    if dist_near not in list_wr:
        new_correct_distance.append(dist_near)
        index_new_correct_distance.append(h)

        
print("Incorrect distance:", len(new_correct_distance),
      "\nIncorrect nearest warehouses:", len(new_correct_warehouse))  

once we use the code, we can tell that there is no mistake in the columns

In [ ]:
anom_dist_wr = list(index_incorrect_distance)
anom_wr = list(set(name_anomalies_wr + index_incorrect_warehouse))

print("Total Incorrect distance:", len(anom_dist_wr),
      "\nTotal Incorrect nearest warehouses:", len(anom_wr))  

we can tell that some of the nearest warehouse have the same index so the number of row that have anomalies in warehouse and the distance are 27

### Handle shopping_cart, order_price, order_total

in this section we will be looking at the `shopping_cart`, `order_price`, `order_total`. To check the anomalies in each row, we will be using at some of the step which is:

* using `np.linalg.solve` to calculate the price of each item in the outliers data
* to calculate the price and check them, we need to use the equation of each item price.
* calculate the total price using 2 different order prices, one of them from the original, others is the price from the each item price.

as you can see, there is no mistake `shopping_cart`, `order_price`, `order_total` in the outliers data. therefore we will use the outliers data to get the each value item price then check every single row in those column.

In [ ]:
# extract the item and the quantity from the quantity using replace and evaluate
outliers_df['item_list'] = outliers_df.shopping_cart\
                .str.replace("(", "[").str.replace(")", "]").apply(eval)
# create empty dictionary
items_warehouse_dict = {}

# create fucntion that calculate the total quantity in each warehouse
def check_type(columns):
    for tup in columns['item_list']:
        if tup[0] not in items_warehouse_dict:
            items_warehouse_dict [tup[0]] = {}
        if columns['nearest_warehouse'] not in items_warehouse_dict [tup[0]]:
            items_warehouse_dict[tup[0]][columns['nearest_warehouse']] = 1
        else:
            items_warehouse_dict[tup[0]][columns['nearest_warehouse']] += 1

# apply the function in every row           
outliers_df[['nearest_warehouse','item_list']].apply(check_type,axis="columns")
items_warehouse_dict

We can see that there are 10 different item in the data. we will create an array that contain the quantity of the item. the array order based on the item from the dictionary in above.

In [ ]:
# create a list for the warehouse
nearest_warehouses = outliers_df.nearest_warehouse.unique().tolist()
# list the items
items_cart = list(items_warehouse_dict.keys())
# extract the item and the quantity from the quantity using replace and evaluate
outliers_df['item_list'] = outliers_df.shopping_cart\
                    .str.replace("(", "[").str.replace(")", "]").apply(eval)

# create a function that create a numpy array of a quantity in the item 
# and order them based on the dict key
def pro_qty(value):
    item_detail = dict(value['item_list'])
    quantity = list(item_detail.values())
    items_product = list(item_detail.keys())
    quantity_list = []
    for n in items_cart:
        if n in items_product:
            quantity_list.append(quantity[items_product.index(n)])
        else:
            quantity_list.append(0)
    return quantity_list

# apply the function to get a new columns
outliers_df['linalg_arr'] = outliers_df.apply(pro_qty,axis=1)
outliers_df[['item_list','linalg_arr']].head()

`linalg_arr` is a column that contain the quantity of the item. we will be using this array to calculate each price of an item using `np.linalg.solve()`. 

In [ ]:
nearest_warehouse = outliers_df.nearest_warehouse.unique().tolist()
# calculate a price for each item 
warehouse_price_item = {}

# loop for every warehouse
for warehouse in nearest_warehouse:
    items = list(items_warehouse_dict.keys())
    
    # take sample based on the number of the item
    data = outliers_df[(outliers_df['nearest_warehouse']==warehouse)]\
                    .sample(n=len(items))
    
    # change the 'linalg_arr' to an array
    variables = np.array(data['linalg_arr'].tolist())
    
    # if the determinant is 0 we will take new sample
    while np.linalg.det(variables) == 0:
        data = outliers_df[(outliers_df['nearest_warehouse']==warehouse)]\
                    .sample(n=len(items))
        variables = np.array(data['linalg_arr'].tolist())
        
    # use np.linalg.solve() to find the linearity change float and round to 2
    price = np.array(data['order_price'].apply(lambda x: float(x)).tolist())
    linear_sol = np.linalg.solve(variables,price).round(2)
    
    # if there is a negative number then resample again
    while sum(1 for number in linear_sol if number < 0) != 0:
        data = outliers_df[(outliers_df['nearest_warehouse']==warehouse)]\
                    .sample(n=len(items))
        variables = np.array(data['linalg_arr'].tolist())
        while np.linalg.det(variables) == 0:
            data = outliers_df[(outliers_df['nearest_warehouse']==warehouse)]\
                    .sample(n=len(items))
            variables = np.array(data['linalg_arr'].tolist())
        price = np.array(data['order_price'].apply(lambda x: float(x)).tolist())
        linear_sol = np.linalg.solve(variables,price).round(2)
        
    # change the result to dictionary
    c = dict(zip(items, linear_sol))
    for i in range(len(items)):
        warehouse_price_item[warehouse] = c
        
print("Price item in Nickolson:", warehouse_price_item['Nickolson'])
print("Price item in Thompson:", warehouse_price_item['Thompson'])
print("Price item in Bakers:", warehouse_price_item['Bakers'])

We can see from the result, in each warehouse they all have the same price. therefore we will use this price to calculate the price and the total. below there will be 2 new different order total:

* `price_total_new`: calulation of the new order price and `coupon_discount` then add with the `delivery_charges`
* `price_total_old`: calulation of the original order price and `coupon_discount` then add with the `delivery_charges`

In [ ]:
# create dot product between the quantity array with the each price
def dot_pro(columns):
    price_result = int(np.dot(np.array(columns['linalg_arr']), linear_sol))
    return price_result

# calculate the total price using the new order price
def calculate_total_new(columns):
    discount = (columns["coupon_discount"]/100)
    total_price_result = (columns["price_result_new"] - 
                          (columns["price_result_new"]*discount))
    total_price_result = total_price_result + columns["delivery_charges"]

    return total_price_result

# calculate the total price using the original price from the dirty data
def calculate_total_old(columns):
    discount = (columns["coupon_discount"]/100)
    total_price_result = (columns["order_price"] -
                          (columns["order_price"]*discount))
    total_price_result = total_price_result + columns["delivery_charges"]
    
    return total_price_result

# apply the function for every row to make a new columns
outliers_df["price_result_new"] = outliers_df.apply(dot_pro, axis="columns")
outliers_df["price_total_new"] = outliers_df.apply(calculate_total_new,
                                                   axis="columns")
outliers_df["price_total_old"] = outliers_df.apply(calculate_total_old,
                                                   axis="columns")

In [ ]:
# use the same code using the dirty data
nearest_warehouses = dirty_df.nearest_warehouse.unique().tolist()
items_cart = list(items_warehouse_dict.keys())


dirty_df['item_list'] = dirty_df.shopping_cart\
    .str.replace("(", "[").str.replace(")", "]").apply(eval)
dirty_df['linalg_arr'] = dirty_df.apply(pro_qty,axis=1)
dirty_df[['item_list','linalg_arr']].head()

In [ ]:
# apply the function for every row to make a new columns
dirty_df["price_result_new"] = dirty_df.apply(dot_pro, axis="columns")
dirty_df["price_total_new"] = dirty_df.apply(calculate_total_new, 
                                             axis="columns")
dirty_df["price_total_old"] = dirty_df.apply(calculate_total_old, 
                                             axis="columns")

after we get the new columns, we will detect the anomalies based on this condition:

`order_price`: if the new order price is not equal with the original order price and new order total same value with the original order total

`order_total`: if the new order price is equal with the original order price and old order total different value with the original order total

`shopping_cart`: if the new order price is not equal with the original order price and old order total same value with the original order total

In [ ]:
# apply the condition in the code
anom_order_price = list(np.where((dirty_df["price_result_new"] != 
                                  dirty_df['order_price']) &
         (dirty_df["price_total_new"] == dirty_df['order_total']))[0])

anom_order_total = list(np.where((dirty_df["price_result_new"] == 
                                  dirty_df['order_price']) &
         (dirty_df["price_total_old"] != dirty_df['order_total']))[0])

anom_shopping_cart = list(np.where((dirty_df["price_result_new"] != 
                                    dirty_df['order_price']) &
         (dirty_df["price_total_old"] == dirty_df['order_total']))[0])

print("incorrect order price:", len(anom_order_price))
print("incorrect order total:", len(anom_order_total))
print("incorrect shopping cart:", len(anom_shopping_cart))

In [ ]:
#  filter the index and change the order price and order total 
dirty_df.loc[anom_order_price,
             "order_price"] = dirty_df.loc[anom_order_price,
                                           "price_result_new"]
dirty_df.loc[anom_order_total,
             "order_total"] = dirty_df.loc[anom_order_total,
                                           "price_total_new"]

As you can see there are 27 anomalies in every columns. Since the `order_price` and `order_total` are fix. we just need to change the item purchases, since the quantity of the item is correct.

to fix the `shopping_cart` we will be using the `permutations()` for every array item's quantity in every columns. After we found the permutations, we will check the value that have the same value with the fix `order_price`

In [ ]:
# may take time to process
items = list(items_warehouse_dict.keys())

# create the function for the permutation
def repair_item_cart(columns):
    permutate = list(permutations(columns["linalg_arr"]))
    
    # loop all the permutation to find the right order price
    for linear_system in permutate:
        check_price = np.dot(np.array(linear_system), linear_sol)
        if check_price == columns["order_price"]:
            break;
            
    # clean the result to get the same format
    list_item_qty = list(zip(items, list(linear_system)))
    result = [(x, y) for x, y in list_item_qty if y != 0]
    return str(result)

# create fucntion to list the keys item
def manipulate_item(column):
    return list(dict(eval(column.replace("[", "").replace("]", ""))).keys())

# apply the function in every columns
repair_item_cart = dirty_df.loc[anom_shopping_cart].apply(repair_item_cart,
                                                          axis = 1)
new_value_item = repair_item_cart.apply(manipulate_item)
old_value_item = dirty_df.loc[anom_shopping_cart]["shopping_cart"]\
                            .apply(manipulate_item)
new_item_series = (new_value_item.apply(set) - old_value_item.apply(set))\
                            .apply(list)
old_item_series = (old_value_item.apply(set)- new_value_item.apply(set))\
                            .apply(list)

# append to list the correct item
correction_item = []
for c in list(dirty_df.loc[anom_shopping_cart]["shopping_cart"].index):
    correction_item.append(dirty_df.loc[anom_shopping_cart]["shopping_cart"][c]\
                        .replace(old_item_series[c][0],new_item_series[c][0]))

# put the result based on the index     
dirty_df.loc[anom_shopping_cart, "shopping_cart"] = np.array(correction_item)
dirty_df['item_list'] = dirty_df.shopping_cart.str.replace("(", "[")\
                    .str.replace(")", "]").apply(eval)
dirty_df['linalg_arr'] = dirty_df.apply(pro_qty, axis=1)

In [ ]:
# check the result using the same code
dirty_df["price_result_new"] = dirty_df.apply(dot_pro, axis="columns")
dirty_df["price_total_new"] = dirty_df.apply(calculate_total_new,
                                             axis="columns")
dirty_df["price_total_old"] = dirty_df.apply(calculate_total_old,
                                             axis="columns")

In [ ]:
# check the result using the same code
correct_order_price = list(np.where((dirty_df["price_result_new"] != 
                                     dirty_df['order_price']) &
         (dirty_df["price_total_new"] == dirty_df['order_total']))[0])
correct_order_total = list(np.where((dirty_df["price_result_new"] == 
                                     dirty_df['order_price']) &
         (dirty_df["price_total_old"] != dirty_df['order_total']))[0])
correct_item_cart = list(np.where((dirty_df["price_result_new"] != 
                                   dirty_df['order_price']) &
         (dirty_df["price_total_old"] == dirty_df['order_total']))[0])

print("incorrect order price:", len(correct_order_price))
print("incorrect order total:", len(correct_order_total))
print("incorrect item cart:", len(correct_item_cart))

In [ ]:
print("Total anomalies row in order price columns:", len(anom_order_price))
print("Total anomalies row in order total columns:", len(anom_order_total))
print("Total anomalies row in shopping cart columns:", len(anom_shopping_cart))

when using the same code, the anomalies is zero

### Handle is_happy_customer

based on the question, to check whether a customer is happy with their last order, the customer's latest review is
classified using a sentiment analysis classifier. `SentimentIntensityAnalyzer` from
`nltk.sentiment.vader` is used to obtain the polarity score. A sentiment is considered positive if
it has a 'compound' polarity score of 0.05 or higher and is considered negative otherwise

In [ ]:
# nltk.download('vader_lexicon')
# load the SentimentIntensityAnalyzer
sentiment_analysis = SentimentIntensityAnalyzer()

# use the code based on the question
dirty_df["predict_customer_review"] = dirty_df["latest_customer_review"]\
    .apply(lambda x : True if sentiment_analysis\
           .polarity_scores(x)["compound"]>0.05 or x == "None" else False)

In [ ]:
anom_review = []

# check the predicted value with the original columns
for i in range(len(dirty_df)):
    if dirty_df["predict_customer_review"][i] != \
        dirty_df["is_happy_customer"][i]:
            anom_review.append(dirty_df["is_happy_customer"][i])
print("Total anomalies row in review columns:", len(anom_review))

In [ ]:
# change the value of the is_happy_customer
dirty_df["is_happy_customer"] = dirty_df["predict_customer_review"]

In [ ]:
correct_review = []
for i in range(len(dirty_df)):
    if dirty_df["predict_customer_review"][i] != \
        dirty_df["is_happy_customer"][i]:
            correct_review.append(dirty_df["is_happy_customer"][i])
print("incorrect review:",len(correct_review))

## 5. Handle Missing Data

Even though `is_expedited_delivery` is still dirty, we will deal with the missing data. Missing data is a data that some value has a `Nan` value this anomalies can be called `Coverage Anomalies` which is decrease the amount of entities and entity properties from the mini-world that are represented in the data collection. Coverage anomalies are categorized as: missing values and missing tuples.

In [ ]:
missing_df.info()

Based on the output in above, there are 500 entries in the data. some columns has missing value, those columns are:

* `nearest_warehouse`:             55 missing data, only have 445 entries out of 500

* `order_price`:                   15 missing data, only have 485 entries out of 500

* `delivery_charges`:              40 missing data, only have 460 entries out of 500
            
* `order_total`:                   15 missing data, only have 485 entries out of 500

* `distance_to_nearest_warehouse`: 31 missing data, only have 469 entries out of 500

* `is_happy_customer`:             40 missing data, only have 460 entries out of 500

In [ ]:
# get the missing index in every columns
dist_warehouse_near_index_miss = list(missing_df[
    missing_df["distance_to_nearest_warehouse"].isnull()].index)
nearest_warehouse_index_miss = list(missing_df[
    missing_df["nearest_warehouse"].isnull()].index)
order_price_index_miss = list(missing_df[
    missing_df["order_price"].isnull()].index)
delivery_charges_index_miss = list(missing_df[
    missing_df["delivery_charges"].isnull()].index)
order_total_index_miss = list(missing_df[
    missing_df["order_total"].isnull()].index)
is_happy_customer_index_miss = list(missing_df[
    missing_df["is_happy_customer"].isnull()].index)

Since we already solve the columns from the dirty data. we will use the the same code to get the value for the missing data. Firstly, we will start from the `distance_to_nearest_warehouse` in the columns we can use the function from the previous cells and use in the missing data and since there is no mistake/error on the missing data we can use other columns as a reference.

In [ ]:
# calculate the distance in each warehouses and put into a new columns
missing_df["distance_to_Nickolson"] = missing_df.apply(
    lambda x : dist_places_warehouse(x["customer_lat"], 
                                     x["customer_long"], 
                                    Nickolson_coor["lat"],
                                    Nickolson_coor["lon"]),
    axis=1).round(4)

missing_df["distance_to_Thompson"] = missing_df.apply(
    lambda x : dist_places_warehouse(x["customer_lat"],
                                     x["customer_long"], 
                                    Thompson_coor["lat"],
                                    Thompson_coor["lon"]), 
    axis=1).round(4)

missing_df["distance_to_Bakers"] = missing_df.apply(
    lambda x : dist_places_warehouse(x["customer_lat"], 
                                     x["customer_long"], 
                                    Bakers_coor["lat"],
                                    Bakers_coor["lon"]), 
    axis=1).round(4)

In [ ]:
# create a empty list
min_nearest = []
correct_warehouse = []
warehouse_places = list(warehouses_df["names"])

# create a loop to find which column has the minimum value
for h in range(len(missing_df)):
    dist_near = round(missing_df["distance_to_nearest_warehouse"][h], 4)
    wr_place = missing_df["nearest_warehouse"][h]
    list_wr = [missing_df["distance_to_Nickolson"][h],
               missing_df["distance_to_Thompson"][h],
               missing_df["distance_to_Bakers"][h]]
    min_dist_idx = list_wr.index(min(list_wr))
    correct_warehouse.append(warehouse_places[min_dist_idx])
    min_nearest.append(min(list_wr))

In [ ]:
# get the value of the missing value
list_miss_warehouse = [min_nearest[i] \
                       for i in dist_warehouse_near_index_miss]
list_miss_distance = [correct_warehouse[i] \
                      for i in nearest_warehouse_index_miss]

missing_df.loc[nearest_warehouse_index_miss,
               "nearest_warehouse"] = list_miss_distance
missing_df.loc[dist_warehouse_near_index_miss,
               "distance_to_nearest_warehouse"] = list_miss_warehouse

Next is for the happy customer, we will use the same build in function which is `SentimentIntensityAnalyzer()` to get the value of the customer happiness.

In [ ]:
# use the sentiment anylyzer 
# from the package to sove the missing data in happy customer
sentiment_analysis = SentimentIntensityAnalyzer()
missing_df.loc[
    is_happy_customer_index_miss, "is_happy_customer"] \
        = missing_df.loc[
        is_happy_customer_index_miss,
            "latest_customer_review"]\
    .apply(lambda x \
           : True if sentiment_analysis.polarity_scores(x)["compound"]>0.05 
           or x == "None" else False)

After we done with the customer happiness we need to fill the `order_price` columns since we already have the price of each item from the previous cells. we will use this price item to calculate the missing price with the shopping cart

In [ ]:
# locate the column based on the missing index
missing_df.loc[order_price_index_miss][["shopping_cart", "order_price"]]

# create an array of the items quantity
missing_df['item_list'] = missing_df.shopping_cart.str.replace("(", "[")\
                            .str.replace(")", "]").apply(eval)
missing_df['linalg_arr'] = missing_df.apply(pro_qty, axis=1)

# calculate the order price value
missing_df.loc[order_price_index_miss, "order_price"] = missing_df\
                .loc[order_price_index_miss].apply(dot_pro, axis="columns")
missing_df["order_price"] = missing_df["order_price"].astype("int64")

for the `delivery_charges` based on the question, The store has different business rules depending on the season to match the different
demands of each season. For example, delivery charge is calculated using a linear model
which differs depending on the season. The model depends linearly (but in different ways for
each season) on:
1. Distance between customer and nearest warehouse
2. Whether the customer wants an expedited delivery
3. Whether the customer was happy with his/her last purchase (if no previous purchase,
it is assumed that the customer is happy)

since we already create the linear mode function from the begining using package called `sklearn` we will fit the model that have non-missing value and predict the missing value.

In [ ]:
# get the missing value and the non-missing value
non_missing_delivery = missing_df[missing_df["delivery_charges"].notnull()]
missing_delivery = missing_df[missing_df["delivery_charges"].isnull()]

In [ ]:
# filter each data based on the season and get the missing index
winter_non_missing = non_missing_delivery[non_missing_delivery['season']
                                          == "Winter"]
winter_missing = missing_delivery[missing_delivery['season'] == "Winter"]
winter_index_miss = list(winter_missing.index)

summer_non_missing = non_missing_delivery[non_missing_delivery['season']
                                          == "Summer"]
summer_missing = missing_delivery[missing_delivery['season'] == "Summer"]
summer_index_miss = list(summer_missing.index)

autumn_non_missing = non_missing_delivery[non_missing_delivery['season']
                                          == "Autumn"]
autumn_missing = missing_delivery[missing_delivery['season'] == "Autumn"]
autumn_index_miss = list(autumn_missing.index)

spring_non_missing = non_missing_delivery[non_missing_delivery['season']
                                          == "Spring"]

spring_missing = missing_delivery[missing_delivery['season'] == "Spring"]
spring_index_miss = list(spring_missing.index)

since there is only 460 correct data in the `delivery_charges` we will use them to fit the data and we will four different model.

In [ ]:
# fit the data for every season
winter_model_missing = linearModel(winter_non_missing)
summer_model_missing = linearModel(summer_non_missing)
autumn_model_missing = linearModel(autumn_non_missing)
spring_model_missing = linearModel(spring_non_missing)

In [ ]:
# print the coeficcient
print(winter_model_missing.coef_, winter_model_missing.intercept_)
print(summer_model_missing.coef_, summer_model_missing.intercept_)
print(autumn_model_missing.coef_, autumn_model_missing.intercept_)
print(spring_model_missing.coef_, spring_model_missing.intercept_)

This Coefficient and intercept we will be used to predict the value for of the `delivery_charges`. this model we will be use in the outliers data and to clean the `is_expedited_delivery` columns

In [ ]:
# predict the rest of the data using the model based on the season
predict_winter_miss = winter_model_missing.predict(winter_missing[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']])
predict_summer_miss = summer_model_missing.predict(summer_missing[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']])
predict_autumn_miss = autumn_model_missing.predict(autumn_missing[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']])
predict_spring_miss = spring_model_missing.predict(spring_missing[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']])

# round the result into 2 decimal
missing_df.loc[winter_index_miss, "delivery_charges"] = predict_winter_miss\
            .round(2)
missing_df.loc[summer_index_miss, "delivery_charges"] = predict_summer_miss\
            .round(2)
missing_df.loc[autumn_index_miss, "delivery_charges"] = predict_autumn_miss\
            .round(2)
missing_df.loc[spring_index_miss, "delivery_charges"] = predict_spring_miss\
            .round(2)

When we done with the missing `delivery_charges`, we will find the missing value for the total of the order

In [ ]:
# calculate the discount
discount_miss = (missing_df.loc[order_total_index_miss,"coupon_discount"]/100)
cal_discount_result = (missing_df.loc[order_total_index_miss,"order_price"] - 
                      (missing_df.loc[order_total_index_miss,"order_price"]
                       *discount_miss))

# get the delivery charges and calculate the missing total
deliv_miss = (missing_df.loc[order_total_index_miss,"delivery_charges"])
missing_df.loc[order_total_index_miss, "order_total"]= cal_discount_result \
                                                    + deliv_miss

In [ ]:
clean_missing_df = missing_df[columns_dirty]
clean_missing_df.info()

we can see that the missing data are all fill with the reasonable value

## 6. Handle Outliers data

In this section  we will be explore the outliers data and detect the outliers in the `delivery_charges` columns. we will use some visualization package such as `seaborn` and `matplotlib` to find the outliers.

In [ ]:
# create a new column that contain zero
outliers_df["predict_delivery"] = 0

In [ ]:
# filter the data based on the season and get the index
winter_outliers = outliers_df[outliers_df['season'] == "Winter"]
winter_index = list(winter_outliers.index)

summer_outliers = outliers_df[outliers_df['season'] == "Summer"]
summer_index = list(summer_outliers.index)

autumn_outliers = outliers_df[outliers_df['season'] == "Autumn"]
autumn_index = list(autumn_outliers.index)

spring_outliers = outliers_df[outliers_df['season'] == "Spring"]
spring_index = list(spring_outliers.index)

In [ ]:
# use the missing data model to predict the value 
missing_predict_winter = list(winter_model_missing.predict(winter_outliers[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']]))

missing_predict_summer = list(summer_model_missing.predict(summer_outliers[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']]))

missing_predict_autumn = list(autumn_model_missing.predict(autumn_outliers[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']]))

missing_predict_spring = list(spring_model_missing.predict(spring_outliers[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']]))

In [ ]:
# put the prediction value to the predict_delivery columns based in the index
outliers_df.loc[winter_index, "predict_delivery"] = missing_predict_winter

outliers_df.loc[summer_index, "predict_delivery"] = missing_predict_summer

outliers_df.loc[autumn_index, "predict_delivery"] = missing_predict_autumn

outliers_df.loc[spring_index, "predict_delivery"] = missing_predict_spring

In [ ]:
# create the new columns to monitor the error
outliers_df["residual"] = (outliers_df["predict_delivery"] - 
                           outliers_df["delivery_charges"])

In [ ]:
# plot the delivery charges columns for each season
sns.boxplot(x="season", y="delivery_charges",
            data=outliers_df)
sns.despine(offset=10, trim=True)

Based on the box plot, there are some value that out of the box. this value will be detect and delete from the data. to do that we will plot the error which the `residual` columns and the error value.

In [ ]:
sns.boxplot(outliers_df.loc[winter_index]["residual"])
plt.show()

In [ ]:
sns.boxplot(outliers_df.loc[summer_index]["residual"])
plt.show()

In [ ]:
sns.boxplot(outliers_df.loc[autumn_index]["residual"])
plt.show()

In [ ]:
sns.boxplot(outliers_df.loc[spring_index]["residual"])
plt.show()

Above is the plot of the error in each season. we can see there are some large error value. This value is from the difference between the prediction and the original. Therfore we will delete the row that have the outliers using the function the we build.

In [ ]:
# https://towardsdatascience.com/a-short-journey-
# of-outlier-detection-bdf143464a92
def detect_outliers_index(columns):
    # calculate the first quantile and third  quantile
    Q1 = columns["residual"].quantile(0.25)
    Q3 = columns["residual"].quantile(0.75)
    IQR = Q3 - Q1
    bool_outliers = (columns["residual"] < (Q1 - 1.5 * IQR)) \
        | (columns["residual"] > (Q3 + 1.5 * IQR))
    return list(bool_outliers[bool_outliers == True].index)

In [ ]:
# use the function for detect the outliers index
outliers_index_winter = detect_outliers_index(outliers_df.loc[winter_index])
outliers_index_summer = detect_outliers_index(outliers_df.loc[summer_index])
outliers_index_autumn = detect_outliers_index(outliers_df.loc[autumn_index])
outliers_index_spring = detect_outliers_index(outliers_df.loc[spring_index])

# total the index
index_list_outliers = outliers_index_winter \
    + outliers_index_summer + outliers_index_autumn \
        + outliers_index_spring

print("outliers detect:", len(index_list_outliers))

In [ ]:
# drop the row based on the index
new_outliers = outliers_df.drop(index_list_outliers).reset_index(drop=True)
new_outliers = new_outliers[columns_dirty]
new_outliers.info()

In [ ]:
sns.boxplot(x="season", y="delivery_charges",
            data=new_outliers)
sns.despine(offset=10, trim=True)

We detect 50 outliers in the data and delete them, then we plot the season and delivery charged again to see if there is another outliers. The data is turn from 500 rows to 450 rows.

## 7. Handle Dirty Is expedited delivery columns

Last problem that need to be clean in the dirty data is the `is_expedited_delivery` columns. the process to solve this problem is:

* Filter the data based on the season and get the index
* use the linear model to predict the value
* use histogram to see the error value and see how many row that have large value error

In [ ]:
dirty_df["is_expedited_delivery"].value_counts()

In [ ]:
# fiter the data based on the season and index
winter_dirty = dirty_df[dirty_df["season"] == "Winter"][[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer', "delivery_charges"]]
winter_dirty_index = list(winter_dirty.index)

summer_dirty = dirty_df[dirty_df["season"] == "Summer"][[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer', "delivery_charges"]]
summer_dirty_index = list(summer_dirty.index)

autumn_dirty = dirty_df[dirty_df["season"] == "Autumn"][[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer', "delivery_charges"]]
autumn_dirty_index = list(autumn_dirty.index)

spring_dirty = dirty_df[dirty_df["season"] == "Spring"][[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer', "delivery_charges"]]
spring_dirty_index = list(spring_dirty.index)

In [ ]:
# predict the delivery charge
winter_predict_dirty = winter_model_missing.predict(winter_dirty[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']])

summer_predict_dirty = summer_model_missing.predict(summer_dirty[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']])

autumn_predict_dirty = autumn_model_missing.predict(autumn_dirty[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']])

spring_predict_dirty = spring_model_missing.predict(spring_dirty[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']])

In [ ]:
# calculate the error and get the data that have more than 5 and less tha -5
winter_error_dirty = winter_dirty["delivery_charges"] - winter_predict_dirty
plt.hist((winter_error_dirty))
plt.show()
index_winter_anom = list(winter_error_dirty[(winter_error_dirty > 5) \
                                            | (winter_error_dirty < -5)].index)

summer_error_dirty = summer_dirty["delivery_charges"] - summer_predict_dirty
plt.hist((summer_error_dirty))
plt.show()
index_summer_anom = list(summer_error_dirty[(summer_error_dirty > 5) \
                                            | (summer_error_dirty < -5)].index)

autumn_error_dirty = autumn_dirty["delivery_charges"] - autumn_predict_dirty
plt.hist((autumn_error_dirty))
plt.show()
index_autumn_anom = list(autumn_error_dirty[(autumn_error_dirty > 5) \
                                            | (autumn_error_dirty < -5)].index)

spring_error_dirty = spring_dirty["delivery_charges"] - spring_predict_dirty
plt.hist((spring_error_dirty))
plt.show()
index_spring_anom = list(spring_error_dirty[(spring_error_dirty > 5) \
                                            | (spring_error_dirty < -5)].index)

Based on the histogram, there are some high error. We will use -5 < error < 5 as a boundary to change the `is_expedited_delivery` where the value is outside of that boundary.

In [ ]:
anom_expe_deliv = len(index_spring_anom) + len(index_autumn_anom) \
                    + len(index_summer_anom) + len(index_winter_anom)
print("Total anomalies for is expedited delivery: ", anom_expe_deliv)

In [ ]:
# create a function to change the value
def expi_deliv(columns):
    if columns["is_expedited_delivery"] == True:
        columns["is_expedited_delivery"] = False
    else:
        columns["is_expedited_delivery"] = True
        
    return columns["is_expedited_delivery"]

# apply the fucntion and put it depend on the index
dirty_df.loc[index_winter_anom, 
             "is_expedited_delivery"] = dirty_df.loc[index_winter_anom]\
                                            .apply(expi_deliv, axis = 1)
dirty_df.loc[index_summer_anom, 
             "is_expedited_delivery"] = dirty_df.loc[index_summer_anom]\
                                            .apply(expi_deliv, axis = 1)
dirty_df.loc[index_autumn_anom, 
             "is_expedited_delivery"] = dirty_df.loc[index_autumn_anom]\
                                            .apply(expi_deliv, axis = 1)
dirty_df.loc[index_spring_anom, 
             "is_expedited_delivery"] = dirty_df.loc[index_spring_anom]\
                                            .apply(expi_deliv, axis = 1)

In [ ]:
# fiter the data based on the season and index
winter_dirty = dirty_df[dirty_df["season"] == "Winter"][[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer', "delivery_charges"]]
winter_dirty_index = list(winter_dirty.index)

summer_dirty = dirty_df[dirty_df["season"] == "Summer"][[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer', "delivery_charges"]]
summer_dirty_index = list(summer_dirty.index)

autumn_dirty = dirty_df[dirty_df["season"] == "Autumn"][[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer', "delivery_charges"]]
autumn_dirty_index = list(autumn_dirty.index)

spring_dirty = dirty_df[dirty_df["season"] == "Spring"][[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer', "delivery_charges"]]
spring_dirty_index = list(spring_dirty.index)

In [ ]:
# predict the delivery charge
winter_predict_dirty = winter_model_missing.predict(winter_dirty[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']])

summer_predict_dirty = summer_model_missing.predict(summer_dirty[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']])

autumn_predict_dirty = autumn_model_missing.predict(autumn_dirty[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']])

spring_predict_dirty = spring_model_missing.predict(spring_dirty[[
    'distance_to_nearest_warehouse',
    'is_expedited_delivery',
    'is_happy_customer']])

In [ ]:
# calculate the error and get the data that have more than 10 and less tha -10
winter_error_dirty = winter_dirty["delivery_charges"] - winter_predict_dirty
plt.hist((winter_error_dirty))
plt.show()
index_winter_new = list(winter_error_dirty[(winter_error_dirty > 5) 
                                           | (winter_error_dirty < -5)].index)

summer_error_dirty = summer_dirty["delivery_charges"] - summer_predict_dirty
plt.hist((summer_error_dirty))
plt.show()
index_summer_new = list(summer_error_dirty[(summer_error_dirty > 5) 
                                           | (summer_error_dirty < -5)].index)

autumn_error_dirty = autumn_dirty["delivery_charges"] - autumn_predict_dirty
plt.hist((autumn_error_dirty))
plt.show()
index_autumn_new = list(autumn_error_dirty[(autumn_error_dirty > 5) 
                                           | (autumn_error_dirty < -5)].index)

spring_error_dirty = spring_dirty["delivery_charges"] - spring_predict_dirty
plt.hist((spring_error_dirty))
plt.show()
index_spring_new = list(spring_error_dirty[(spring_error_dirty > 5) 
                                           | (spring_error_dirty < -5)].index)

In [ ]:
dirty_df["is_expedited_delivery"].value_counts()

We can see that the `is_expedited_delivery` is already clean and ready to be export.

## 8. Export Data

We done with clean all the anomalies in the data, we just need to export the data into a `.csv` file

In [ ]:
clean_df = dirty_df[columns_dirty]
clean_df.info()

clean_df.to_csv("30399262_dirty_data_solution.csv", index = False)

In [ ]:
clean_missing_df.info()

clean_missing_df.to_csv("30399262_missing_data_solution.csv", index = False)

In [ ]:
new_outliers.info()
new_outliers.to_csv("30399262_outlier_data_solution.csv", index = False)

In [ ]:
total_anom_row = anom_date + len(anom_dist_wr) +\
    anom_expe_deliv +len(anom_coor)+ len(anom_order_price)+\
    len(anom_order_total)+\
    len(anom_review)+len(anom_season)+len(anom_shopping_cart)+len(anom_wr)
print("Total Anomalies Row:", total_anom_row)

## 9. Summary

This assignment involves the knowledge on exploratory data analysis, outliers detection and removal, anomalies detection and correction, and data imputation techniques in Python programming language using libraries such as sklearn, re, numpy, seaborn, nltk, pandas, ittertools. The steps encompasses the following processes :


- **Detect and clean dirty data**. By using pandas dataframe and seaborn to do exploratory data analysis, ntlk to clean the review and use np.linalg to get the each item price then find the anomalies in the shopping cart and order total. Then use visualization to get the coordinate and calculate the distance. lastly, use sklearn to perform the linear model in the is expedited delivery.

- **Detect and impute missing values**. using the same process like clean the data we manange to fill the missing value in every columns that still have nan value

- **Outliers detection and removal**. By using basic pandas dataframe manipulation and seaborn to generate boxplot.
- **Observe the correct data in outlier data**. By numpy to calculate linear calculation, sklearn to generate linear model from other table to get the prediction, and pandas dataframe manipulation.

## Reference

- Geeksforgeeks. (2020, April 20). *Python program to swap two elements in a list* Retrieved from https://www.geeksforgeeks.org/python-program-to-swap-two-elements-in-a-list/

- Australian Government (2020, October 14). *Climate Glossary* Retrieved from http://www.bom.gov.au/climate/glossary/seasons.shtml

- smooth007. (2015, October 9). *Using Pandas to calculate distance between coordinates from imported csv* Retrieved from https://stackoverflow.com/questions/33029396/using-pandas-to-calculate-distance-between-coordinates-from-imported-csv

- Ryota Bannai (2019, January 13). *A short journey of outlier detection* Retrieved from https://towardsdatascience.com/a-short-journey-of-outlier-detection-bdf143464a92

- Numpy (2020, October 14). *numpy.linalg.solve* Retrieved from https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html

- NLTK 3.5 documentation (2020, October 14). *nltk.sentiment.sentiment_analyzer module* Retrieved from https://www.nltk.org/api/nltk.sentiment.html#nltk.sentiment.vader.SentimentIntensityAnalyzer.score_valence

- Romaine Herrera (2014, February 27). *Regex date validation for yyyy-mm-dd [duplicate]* Retrieved from https://stackoverflow.com/questions/22061723/regex-date-validation-for-yyyy-mm-dd

- Python document (2020, October 14) *itertools — Functions creating iterators for efficient looping* Retrieved from https://docs.python.org/2/library/itertools.html

- user1654183 (2013, May 2) *Convert a Pandas DataFrame to a dictionary* Retrieved from https://stackoverflow.com/questions/16343752/numpy-where-function-multiple-conditions